In [1]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [4]:
def research(Qid):
    sparql.setQuery("""
    SELECT ?item ?itemLabel
    WHERE 
    {
      ?item wdt:P31 wd:""" + Qid + """. #on pourais rajouté des obligation de ref rance a des payes etc
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # le label viendra de préférence dans votre langue, et autrement en anglais
      
    }LIMIT 1000
    """)
    #/wdt:P31/wdt:P279
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        # ne prend pas ce ou il y a pas de nom ajouté
        if(result["itemLabel"]["value"][0]!="Q"):                                   
            mydict.append(result["itemLabel"]["value"])
            
    return mydict

In [5]:
listofidqHuman = ['Q22808320', 'Q9415', 'Q31629', 'Q188451', 'Q192581', 'Q483394', 'Q11410', 'Q2095', 'Q223557', 'Q3624078']


In [6]:
description = [research(listofidqHuman[0]),research(listofidqHuman[1]),research(listofidqHuman[2]),research(listofidqHuman[3]),research(listofidqHuman[4])
               ,research(listofidqHuman[5]),research(listofidqHuman[6]),research(listofidqHuman[7]),research(listofidqHuman[8]),research(listofidqHuman[9])]

In [7]:
!pip install transformers==3.3.1
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [8]:
model_path = "./generation model/"

In [9]:
tokenizer = T5Tokenizer.from_pretrained(model_path)
model =T5ForConditionalGeneration.from_pretrained(model_path, return_dict=True)

In [10]:
def generate(text,model,tokenizer):
    model.eval()
    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  
    outputs = model.generate(input_ids, max_length=300, temperature=0.1)
    return tokenizer.decode(outputs[0])

In [11]:
import unicodedata

In [12]:
import gensim

# Load Google's pre-trained Word2Vec model.
word2vecModel = gensim.models.KeyedVectors.load_word2vec_format('./word2vec model/GoogleNews-vectors-negative300.bin', binary=True) 

In [57]:
#topics contient les sujets pour lesquels il y a des méthodes pour le dialogue
#quand on implémente des nouvelles méthodes il faut rajouter le sujet ici
topics=['music', 'job', 'food', 'game', 'sport']

IT manager


In [94]:
class Person:
    def __init__(self):
        self.age = random.randint(18, 80)
        self.name = unicodedata.normalize('NFD', random.choice(description[0])).encode('ascii', 'ignore').decode('utf-8')
        self.emotion = unicodedata.normalize('NFD', random.choice(description[1])).encode('ascii', 'ignore').decode('utf-8')
        self.sport = unicodedata.normalize('NFD', random.choice(description[2])).encode('ascii', 'ignore').decode('utf-8')
        self.music = unicodedata.normalize('NFD', random.choice(description[3])).encode('ascii', 'ignore').decode('utf-8')
        self.job = unicodedata.normalize('NFD', random.choice(description[4])).encode('ascii', 'ignore').decode('utf-8')
        self.genre =  unicodedata.normalize('NFD', random.choice(description[5])).encode('ascii', 'ignore').decode('utf-8')
        self.game =  unicodedata.normalize('NFD', random.choice(description[6])).encode('ascii', 'ignore').decode('utf-8')
        self.food =  unicodedata.normalize('NFD', random.choice(description[7])).encode('ascii', 'ignore').decode('utf-8')
        self.objectOfInterest =  unicodedata.normalize('NFD', random.choice(description[8])).encode('ascii', 'ignore').decode('utf-8')
        self.country = unicodedata.normalize('NFD', random.choice(description[9])).encode('ascii', 'ignore').decode('utf-8')

    def generateDescription(self):
        surname=self.name.split(' ')[0]
        string=self.name+' | age | '+str(self.age)+' && '+self.name+' | country | '+self.country
        print(generate(string, model, tokenizer))
        string=surname+' | job | '+self.job
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteSport | '+self.sport
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteMusic | '+self.music
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteGenre | '+self.genre
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteGame | '+self.game
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFood | '+self.food
        print(generate(string, model, tokenizer))
        string=surname+' | interestedIn | '+self.objectOfInterest
        print(generate(string, model, tokenizer))

    def showProperties(self):
        print('name: '+self.name+'\nage: '+str(self.age)+'\ncountry: '+self.country+'\nemotion: '+self.emotion+'\nmusic: '+self.music+'\njob: '+self.job+'\nsport: '+self.sport+'\ngenre: '+self.genre+'\ngame: '+self.game+'\nfood: '+self.food+'\nobject of interest: '+self.objectOfInterest+'\n')
        
    def introduce(self):
        text=self.name.split(' ')[0]+':   '

        #Greeting
        rand=random.randint(0,3)
        if rand==0:
            text+='Hello, '
        elif rand==1:
            text+='Hi, '
        elif rand==2:
            text+='Hey, '
        elif rand==3:
            text+='Good day, '
      
        #Name
        rand=random.randint(0,2)
        if rand==0:
            text+="I'm "
        elif rand==1:
            text+="I am "
        elif rand==2:
            text+="my name is "
        text+=self.name+', '

        #Age
        rand=random.randint(0,1)
        if rand==0:
            text+='I am '
        elif rand==1:
            text+="I'm "
        text+=str(self.age)+' years old '

        #Country
        rand=random.randint(0,3)
        if rand==0:
            text+="and I'm from "
        elif rand==1:
            text+="and I am from "
        elif rand==2:
            text+="and I come from "
        elif rand==3:
            text+="and I live in "
        text+=self.country+'.'

        print(text)

    def tellYourFavouriteFood(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite food is '
        elif rand==1:
            text+='I like to eat '
        elif rand==2:
            text+='I love '
        elif rand==3:
            text+='My favourite dish is '
        elif rand==4:
            text+='I really enjoy '
        text+=self.food+'.'
        return text

    def startTopicFood(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFood()+' '
        #Ask the others about their favourite food
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite food?'
        elif rand==1:
            text+='What do you like to eat most?'
        elif rand==2:
            text+='What is your preferred dish?'
        elif rand==3:
            text+='What is your favourite dish?'
        elif rand==4:
            text+='Which food do you enjoy most?'
        print(name+text)
        return text

    def respondToFood(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteFood()
        print(text)

    def tellYourJob(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='I work as a '
        elif rand==1:
            text+='I am '
        elif rand==2:
            text+="I'm "
        elif rand==3:
            text+='My job is '
        elif rand==4:
            text+='I make a living from working as a '
        text+=self.optimizeJob(self.job)+'.'
        return text
    
    def optimizeJob(self, job):
        split=job.split(' ')
        for i in range(len(split)):
            if split[i]=='management':
                split[i]='manager'
            elif split[i]=='support':
                split[i]='supporter'
        optimizedJob=''
        for word in split:
            optimizedJob+=word+' '
        optimizedJob=optimizedJob[:len(optimizedJob)-1]
        return optimizedJob

    def startTopicJob(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourJob()+' '
        #Ask the others about their job
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your job?'
        elif rand==1:
            text+='What are you doing for a living?'
        elif rand==2:
            text+='What are you working?'
        elif rand==3:
            text+='What do you do?'
        elif rand==4:
            text+='What do you work as?'
        print(name+text)
        return text

    def respondToJob(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourJob()
        print(text)

    def tellYourFavouriteMusic(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite music is '
        elif rand==1:
            text+='I like listening to '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='My favourite music genre is '
        elif rand==4:
            text+='I really enjoy listening to '
        elif rand==5:
            text+='I usually listen to '
        text+=self.music+'.'
        return text

    def startTopicMusic(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteMusic()+' '
        #Ask the others about their favourite music
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite music?'
        elif rand==1:
            text+='What do you like to listen to?'
        elif rand==2:
            text+='What is your favourite music genre?'
        elif rand==3:
            text+='What music do you like to listen to?'
        elif rand==4:
            text+='Which type of music do you enjoy most?'
        print(name+text)
        return text

    def respondToMusic(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteMusic()
        print(text)
        
    def tellYourFavouriteGame(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite game is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        text+=self.game+'.'
        return text

    def startTopicGame(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteGame()+' '
        #Ask the others about their favourite game
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite game?'
        elif rand==1:
            text+='What do you like to play?'
        elif rand==2:
            text+='Which games do you play?'
        elif rand==3:
            text+='Are there games that you like to play?'
        elif rand==4:
            text+='Which game do you enjoy most?'
        print(name+text)
        return text

    def respondToGame(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteGame()
        print(text)
        
    def tellYourFavouriteSport(self):
        text=''
        rand=random.randint(0,9)
        if rand==0:
            text+='My favourite sport is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        elif rand==6:
            text+='I like watching '
        elif rand==7:
            text+='I like to watch '
        elif rand==8:
            text+='I\'m a fan of '
        elif rand==9:
            text+='I\'m a '+self.sport+' fan.'
        if not rand==9:
            text+=self.sport+'.'
        return text

    def startTopicSport(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteSport()+' '
        #Ask the others about their favourite sport
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite sport?'
        elif rand==1:
            text+='Are you a sports fan?'
        elif rand==2:
            text+='Which sport are you interested in?'
        elif rand==3:
            text+='What is your preferred sport?'
        elif rand==4:
            text+='Which kind of sport do you like most?'
        print(name+text)
        return text

    def respondToSport(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteSport()
        print(text)

    def respond(self, message):
        split=message.split('. ')
        statement=split[0]
        question=split[1]
        question=question[:len(question)-1]
        guessedTopic=self.guessTopic(question)
        SWITCH_DICT={
          #'emotion':self.respondToEmotion,
          'sport':self.respondToSport,
          'music':self.respondToMusic,
          'job':self.respondToJob,
          #'genre':self.respondToGenre,
          'game':self.respondToGame,
          'food':self.respondToFood,
          #'objectOfInterest':self.respondToObjectOfInterest
        }
        SWITCH_DICT.get(guessedTopic)()
        
    def guessTopic(self, text):
        bestTopic=None
        bestDistance=None
        words=text.split(' ')
        for word in words:
            for topic in topics:
                try:
                    distance=word2vecModel.distance(word, topic)
                    if bestTopic==None or distance<bestDistance:
                        bestTopic=topic
                        bestDistance=distance
                except:
                    continue
        return bestTopic
    
    def startTopic(self, doneTopics):
        openTopics=[topic for topic in topics if topic not in doneTopics]
        if len(openTopics)==0:
            return None
        rand=random.randint(0,len(openTopics)-1)
        if openTopics[rand]=='music':
            doneTopics.append('music')
            return self.startTopicMusic()
        elif openTopics[rand]=='job':
            doneTopics.append('job')
            return self.startTopicJob()
        elif openTopics[rand]=='food':
            doneTopics.append('food')
            return self.startTopicFood()
        elif openTopics[rand]=='game':
            doneTopics.append('game')
            return self.startTopicGame()
        elif openTopics[rand]=='sport':
            doneTopics.append('sport')
            return self.startTopicSport()
        else:
            return None

In [17]:
def generatePersons(nPersons):
    persons=[]
    for i in range(nPersons):
        person = Person()
        persons.append(person)
    return persons

In [18]:
def startChat(persons, nTopics):
    doneTopics=[]
    for person in persons: 
          person.introduce()
    for i in range(nTopics):
        personToStart=persons[random.randint(0, len(persons)-1)]
        message=personToStart.startTopic(doneTopics)
        if message==None:
            return
        random.shuffle(persons)
        for person in persons:
            if not person==personToStart:
                person.respond(message)

In [19]:
def generateDescriptions(persons):
    for person in persons:
        person.generateDescription()
        print('\n')

In [99]:
myPersons=generatePersons(4)

In [100]:
startChat(myPersons, 5)           

Brian:   Hi, I am Brian Adams, I am 41 years old and I live in Crown of Aragon.
Thomas:   Hey, my name is Thomas Wentworth, I'm 59 years old and I come from New Zealand.
Harry:   Hello, I'm Harry Armstrong, I'm 45 years old and I come from Latvia.
Bill:   Hey, I'm Bill Nelson, I am 38 years old and I come from Dominican Republic.
Thomas:   My favourite game is The Fungi from Yuggoth. What do you like to play?
Harry:   I like playing riquitillas.
Brian:   My favourite game is Deadzone.
Bill:   I like Power hour.
Brian:   I like watching British baseball. What is your preferred sport?
Thomas:   I like watching sport climbing.
Harry:   I like What? Where? When?.
Bill:   I'm a fan of sled dog racing.
Brian:   I like zouglou. What do you like to listen to?
Harry:   I like post-bop.
Thomas:   My favourite music is music of Hungary.
Bill:   My favourite music genre is Changgeuk.
Bill:   I make a living from working as a trunk maker. What do you work as?
Thomas:   I am sales position.
Harry:  

In [61]:
generateDescriptions(myPersons)

C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Jitender Kumar, who is 67 years old, is from Colombia.
The honorary consul role for Jitender is honorary consul.
The memory sport of a Jitender is the memory sport.
The favourite of aitender is 2 tone.
The political leader of the country of Jitender is the political leader.
The favourite of aitender is the Wingspan.
Sanuki udon is a favourite of aitender.
The 30 year long study at the ULB, led by Leon Vandenkindere, is interested in the study of Jitender.


David Johnson, who is a member of the Islamic Emirate of Afghanistan, has a career of 33.
The leader of David is a magistrate.
Teqvoly is a favourite sport of David.
The favourite of David is Taitse Opera.
The favourite genre of David is Uepeker.
Buzzword is a favourite of the David series.
The favourite food of David is rawon.
The wizard of David is Speed and Magic.


The Ivan the Terrible is in Barbados and has a total of 5 years.
The nihon-buyo dancer, Ivan, performed as a test pilot.
The favourite sport of Ivan is wrestling.
The

In [62]:
for person in myPersons:
    person.showProperties()

name: Jitender Kumar
age: 67
country: Colombia
emotion: gloom
music: 2 tone
job: honorary consul
sport: memory sport
genre: political talk show
game: Wingspan
food: Sanuki udon
object of interest: 30 years of teaching by Leon Vandenkindere at the ULB

name: David Johnson
age: 33
country: Islamic Emirate of Afghanistan
emotion: Mudita
music: Taiwanese opera
job: dictator
sport: Teqvoly
genre: Uepeker
game: buzzword bingo
food: rawon
object of interest: Speed and Magic

name: Ivan the Terrible
age: 56
country: Barbados
emotion: annoyance
music: fanfare
job: nihon-buyo dancer
sport: wrestling
genre: Conceptual writing
game: Monty Python and the Holy Grail Collectible Card Game
food: Cumberland sausage
object of interest: tapestry

name: Aaron Johnson
age: 33
country: Uganda
emotion: infatuation
music: turbo-folk
job: Theriogenologist
sport: equitation
genre: personal testimonial
game: Three player mahjong
food: rolex
object of interest: Centenary of the Royal Philanthropic Society



In [ ]:
def a():
    text=input()